## Setup and Import Libraries

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

## FAISS

Facebook AI Similarity Search (FAISS) is a library for efficient similiarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibily do not fit in RAM. It also contains supporting code for evaluation and parameter tuning

### Text Splitting

In [4]:
loader = TextLoader(file_path='speech.txt')
documents = loader.load()

In [5]:
text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=30
)

In [7]:
docs = text_splitter.split_documents(documents)
docs

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 670, which is longer than the specified 200
Created a chunk of size 984, which is longer than the specified 200
Created a chunk of size 791, which is longer than the specified 200


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

### Embeddings and Vector Store

In [9]:
embeddings = (
    OllamaEmbeddings(model='gemma:2b')  ## By Default = llama2
)

In [15]:
db = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)
db

### Querying Vector Database

In [16]:
query1 = "What does the speaker believe is the main reason the United States should enter the war?"
query2 = "How does the speaker describe the desired outcome of the war?"

In [17]:
result = db.similarity_search(query=query1)
print(result[0].page_content)

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.


In [18]:
result = db.similarity_search(query=query2)
print(result[0].page_content)

Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.


### Retriever

In [20]:
retriever = db.as_retriever()
retriever_results = retriever.invoke(query1)

In [21]:
retriever_results[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

### Similarity Search with Score

There are some FAISS specific methods. One of them is similarity search with score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better

In [22]:
docs_and_score = db.similarity_search_with_score(query=query1)
docs_and_score

[(Document(id='8db46ab1-3c9a-4f9c-a440-6ccd41d3447c', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  0.5498823),
 (Document(id='e30b40cc-8c5b-4c59-877a-24e92d4de6b0', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friend

In [23]:
docs_and_score[0]

(Document(id='8db46ab1-3c9a-4f9c-a440-6ccd41d3447c', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 0.5498823)

### Passing the Embeddings

In [24]:
embedding_vector = embeddings.embed_query(text=query1)
results = db.similarity_search_by_vector(embedding=embedding_vector)
results

[Document(id='8db46ab1-3c9a-4f9c-a440-6ccd41d3447c', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='e30b40cc-8c5b-4c59-877a-24e92d4de6b0', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German 

### Saving the Vector Database

In [26]:
db.save_local('faiss_index')

### Loading the Vector Database

In [28]:
new_db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [29]:
result = new_db.similarity_search(query=query1)
result

[Document(id='8db46ab1-3c9a-4f9c-a440-6ccd41d3447c', metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='e30b40cc-8c5b-4c59-877a-24e92d4de6b0', metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German 